## **Initialization**

In [0]:
!pip install -U ipykernel

Requirement already up-to-date: ipykernel in /usr/local/lib/python3.6/dist-packages (5.1.4)


In [0]:
!pip install python-telegram-bot --upgrade

Requirement already up-to-date: python-telegram-bot in /usr/local/lib/python3.6/dist-packages (12.3.0)


In [0]:
!pip install PILLOW
!pip install scipy==1.0

In [0]:
from io import BytesIO
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from scipy import misc
import numpy as np
import copy

## **Style Transfer**

### *Content and Style losses (and Normalization here)*

In [0]:
class ContentLoss(nn.Module):

        def __init__(self, target,):
            super(ContentLoss, self).__init__()
            # we 'detach' the target content from the tree used
            # to dynamically compute the gradient: this is a stated value,
            # not a variable. Otherwise the forward method of the criterion
            # will throw an error.
            self.target = target.detach()#это константа. Убираем ее из дерева вычеслений
            self.loss = F.mse_loss(self.target, self.target )#to initialize with something

        def forward(self, input):
            self.loss = F.mse_loss(input, self.target)
            return input

In [0]:
 def gram_matrix(input):
        batch_size , h, w, f_map_num = input.size()  # batch size(=1)
        # b=number of feature maps
        # (h,w)=dimensions of a feature map (N=h*w)

        features = input.view(batch_size * h, w * f_map_num)  # resise F_XL into \hat F_XL

        G = torch.mm(features, features.t())  # compute the gram product

        # we 'normalize' the values of the gram matrix
        # by dividing by the number of element in each feature maps.
        return G.div(batch_size * h * w * f_map_num)

In [0]:
class StyleLoss(nn.Module):
        def __init__(self, target_feature):
            super(StyleLoss, self).__init__()
            self.target = gram_matrix(target_feature).detach()
            self.loss = F.mse_loss(self.target, self.target)# to initialize with something

        def forward(self, input):
            G = gram_matrix(input)
            self.loss = F.mse_loss(G, self.target)
            return input

In [0]:
class Normalization(nn.Module):
        def __init__(self, mean, std):
            super(Normalization, self).__init__()
            # .view the mean and std to make them [C x 1 x 1] so that they can
            # directly work with image Tensor of shape [B x C x H x W].
            # B is batch size. C is number of channels. H is height and W is width.
            self.mean = torch.tensor(mean).view(-1, 1, 1)
            self.std = torch.tensor(std).view(-1, 1, 1)

        def forward(self, img):
            # normalize img
            return (img - self.mean) / self.std

### *class StyleTrandferModel():*

In [0]:
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

In [0]:
class StyleTransferModel():
    def __init__(self, content_layers=content_layers_default, style_layers=style_layers_default, num_steps = 500, style_weight = 100000):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.cnn = models.vgg19(pretrained=True).features.to(self.device).eval()
        self.imsize = 128  
        self.loader = transforms.Compose([
            transforms.Resize(self.imsize),  # нормируем размер изображения
            transforms.CenterCrop(self.imsize),
            transforms.ToTensor()])
        self.unloader = transforms.ToPILImage()
        self.normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(self.device)
        self.normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(self.device)
        self.content_layers = content_layers
        self.style_layers = style_layers

        self.num_steps=num_steps
        self.style_weight=style_weight
        self.content_weight=1

        pass

    def image_loader(self, image_stream):
        image = Image.open(image_stream)
        image = self.loader(image).unsqueeze(0)

        return image.to(self.device, torch.float)

    def imshow(self, tensor, title=None, gr = False):
        image = tensor.cpu().clone()   
        image = image.squeeze(0)      # функция для отрисовки изображения
        image = self.unloader(image)
        plt.imshow(image)
        if gr == True:
            if title is not None:
                plt.title(title)
            plt.pause(0.001)

    def get_style_model_and_losses(self, style_img, content_img):
        cnn = copy.deepcopy(self.cnn)

        # normalization module
        normalization = Normalization(self.normalization_mean, self.normalization_std).to(self.device)

        # just in order to have an iterable access to or list of content/syle
        # losses
        content_losses = []
        style_losses = []

        # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
        # to put in modules that are supposed to be activated sequentially
        model = nn.Sequential(normalization)

        i = 0  # increment every time we see a conv
        for layer in cnn.children():
            if isinstance(layer, nn.Conv2d):
                i += 1
                name = 'conv_{}'.format(i)
            elif isinstance(layer, nn.ReLU):
                name = 'relu_{}'.format(i)
                # The in-place version doesn't play very nicely with the ContentLoss
                # and StyleLoss we insert below. So we replace with out-of-place
                # ones here.
                #Переопределим relu уровень
                layer = nn.ReLU(inplace=False)
            elif isinstance(layer, nn.MaxPool2d):
                name = 'pool_{}'.format(i)
            elif isinstance(layer, nn.BatchNorm2d):
                name = 'bn_{}'.format(i)
            else:
                raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

            model.add_module(name, layer)

            if name in self.content_layers:
                # add content loss:
                target = model(content_img).detach()
                content_loss = ContentLoss(target)
                model.add_module("content_loss_{}".format(i), content_loss)
                content_losses.append(content_loss)

            if name in self.style_layers:
                # add style loss:
                target_feature = model(style_img).detach()
                style_loss = StyleLoss(target_feature)
                model.add_module("style_loss_{}".format(i), style_loss)
                style_losses.append(style_loss)

        # now we trim off the layers after the last content and style losses
        #выбрасываем все уровни после последенего styel loss или content loss
        for i in range(len(model) - 1, -1, -1):
            if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
                break

        model = model[:(i + 1)]

        return model, style_losses, content_losses

    def get_input_optimizer(self, input_img):
        # this line to show that input is a parameter that requires a gradient
        #добоваляет содержимое тензора катринки в список изменяемых оптимизатором параметров
        optimizer = optim.LBFGS([input_img.requires_grad_()]) 
        return optimizer

    

    def transfer_style(self, content_img_stream, style_img_stream):
        style_img = self.image_loader(style_img_stream)
        content_img = self.image_loader(content_img_stream)
        input_img = content_img.clone()

        """Run the style transfer."""
        print('Building the style transfer model..')
        model, style_losses, content_losses = self.get_style_model_and_losses(style_img, content_img)
        optimizer = self.get_input_optimizer(input_img)

        print('Optimizing..')
        run = [0]
        while run[0] <= self.num_steps:

            def closure():
                # correct the values 
                # это для того, чтобы значения тензора картинки не выходили за пределы [0;1]
                input_img.data.clamp_(0, 1)

                optimizer.zero_grad()

                model(input_img)

                style_score = 0
                content_score = 0

                for sl in style_losses:
                    style_score += sl.loss
                for cl in content_losses:
                    content_score += cl.loss
                
                #взвешивание ощибки
                style_score *= self.style_weight
                content_score *= self.content_weight

                loss = style_score + content_score
                loss.backward()

                run[0] += 1
                if run[0] % 50 == 0:
                    print("run {}:".format(run))
                    print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                        style_score.item(), content_score.item()))
                    print()

                return style_score + content_score

            optimizer.step(closure)

        # a last correction...
        #print('imput_image before last correction', input_img.size())
        input_img.data.clamp_(0, 1)
        #print('imput_image after last correction and with .data', input_img.data.size())


        return misc.toimage(input_img.data.squeeze())

## **Telegram Bot**

In [0]:
from telegram.ext import Updater, MessageHandler, Filters, CommandHandler, ConversationHandler
import logging

In [0]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

logger = logging.getLogger(__name__)

In [0]:
model = StyleTransferModel(num_steps = 500, style_weight = 100000)
first_image_file = {}

In [0]:
PHOTO = 0

In [0]:
def start(update, context):
    update.message.reply_text(
        "Hey, hey! Welcome to my Lab. I'm Professor Mutator.\n"
        'Ok, let me show you something.\n\n'
        
        'Send me a Content photo')
    return PHOTO

In [0]:
def send_prediction_on_photo(update, context):
    # Нам нужно получить две картинки, чтобы произвести перенос стиля, но каждая картинка приходит в
    # отдельном апдейте, поэтому в простейшем случае мы будем сохранять id первой картинки в память,
    # чтобы, когда уже придет вторая, мы могли загрузить в память уже сами картинки и обработать их.
    # Точно место для улучшения, я бы
    phrases = ["Super cool!", "I absolutely like that one!", "Great choice!", 
               "Wow! That's a realy great one.", "This one is amazing.", 
               "That's kind of sophisticated photo.", "I'm surprised at you!", 
               "Oh là là...", "I'm crazy about that one!", "I like it!",
               "Awesome!", "Oh, wow!", "Not bad, absolutely not bad...",
               ]

    Puzzles = ["Choose two numbers, x and y, each uniformly from [0,1]. What's the probability x/y rounds to an even number?"
               ]

    random_phrase = phrases[int(np.random.uniform(0,1000))% len(phrases)]

    random_puzzle = Puzzles[int(np.random.uniform(0,1000))% len(Puzzles)]

    chat_id = update.message.chat_id
    print("Got image from {}".format(chat_id))

    # получаем информацию о картинке
    image_info = update.message.photo[-1]
    image_file = context.bot.get_file(image_info)

    if chat_id in first_image_file:
        # первая картинка, которая к нам пришла станет content image, а вторая style image
        update.message.reply_text(random_phrase + "\n\nLet's see the result.\n"
                                                  "Oh, but it will take some time...\n"
                                                  "About 5 minutes \n"
                                                  "You can go and have a cup of tea or coffee (I have no idea what you prefer) or you can try to solve that puzzle:\n\n "
                                                  + random_puzzle)
        content_image_stream = BytesIO()
        first_image_file[chat_id].download(out=content_image_stream)
        del first_image_file[chat_id]

        style_image_stream = BytesIO()
        image_file.download(out=style_image_stream)
        output = model.transfer_style(content_image_stream, style_image_stream)

        # теперь отправим назад фото
        output_stream = BytesIO()
        output.save(output_stream, format='PNG')
        output_stream.seek(0)
        context.bot.send_photo(chat_id, photo=output_stream)
        print("Sent Photo to user")
    else:
        first_image_file[chat_id] = image_file
        update.message.reply_text(random_phrase + ' Now send me a style photo.')

In [0]:
def cancel(update, context):
    user = update.message.from_user
    logger.info("User %s do not have enough time to visit Mutation Laboratory.", user.first_name)
    update.message.reply_text('Bye! I hope we can talk again some day.')
    
    return ConversationHandler.END

def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)

In [0]:
def main():    
    updater = Updater(token="TOKEN", use_context=True)
    dp = updater.dispatcher

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],

        states={
            PHOTO: [MessageHandler(Filters.photo, send_prediction_on_photo)],
        },
        fallbacks=[CommandHandler('cancel', cancel)]
    )

    # # dp.add_handler(CommandHandler("start", start))
    # dp.add_handler(MessageHandler(Filters.photo, send_prediction_on_photo))
    dp.add_handler(conv_handler)

    dp.add_error_handler(error)

    updater.start_polling()
    updater.idle()
if __name__ == '__main__':
    main()

2020-02-08 12:13:20,831 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2020-02-08 12:13:21,749 - telegram.ext.updater - WARNING - Exiting immediately!


## **SGRAN**